## Train a Universal Background Model
UBM模型可以使用Mixture类通过EM算法训练得到

### 1. EM_split训练

In [ ]:
ubm = sidekit.Mixture()

ubm.EM_split(features_server=fs,
             feature_list=ubm_list,
             distrib_nb=1024,
             iterations=(1, 2, 2, 4, 4, 4, 4, 8, 8, 8, 8, 8, 8),
             num_thread=10,
             # 是否每次迭代后保存模型
             save_partial=False,
             ceil_cov=10,
             floor_cov=1e-2
             )

训练1024个分布：  
具有1个分布的1次EM迭代  
具有2个分布的2次EM迭代  
具有4个分布的2次EM迭代  
具有8个分布的4次EM迭代  
具有16个分布的4次EM迭代  
具有32个分布的4次EM迭代  
具有64个分布的4次EM迭代  
具有128个分布的8次EM迭代  
具有256个分布的EM的8次迭代  
具有512个分布的8次EM迭代  
具有1024个分布的8次EM迭代  

### 2. EM_uniform固定数量分布训练
通过直接设置分布数量，也可以通过EM算法训练GMM。在这种情况下，将通过获取训练数据的随机子集的均值和协方差来初始化分布。代码如下：

ubm = sidekit.Mixture()

ubm.EM_uniform(cep,
       distrib_nb,
       iteration_min=3,
       iteration_max=10,
       llk_gain=0.01,
       do_init=True
       )

### 3. EM_split多个节点上训练
SIDEKIT允许通过消息传递接口（MPI）使用多个节点（机器）并行训练GMM。

In [ ]:
ubm = sidekit.Mixture()
sidekit.sidekit_mpi.EM_split(ubm=ubm,
                             features_server=fs,
                             feature_list=ubm_list,
                             distrib_nb=2048,
                             output_filename="ubm_tandem_mpi",
                             iterations=(1, 2, 2, 4, 4, 4, 4, 8, 8, 8, 8, 8, 8, 8, 8),
                             llk_gain=0.01,
                             save_partial=True,
                             ceil_cov=10,
                             floor_cov=1e-2,
                             num_thread=30)

fs是FeaturesServer用于加载声学特征的对象  
ubm_list 是要处理的节目（会话）列表  

### 4. 完全协方差 
为了训练完全协方差GMM，您可以首先使用上述两种方法之一训练具有对角协方差的GMM，然后执行多次EM迭代以估计完整的协方差矩阵。这可以实现如下：

In [ ]:
ubm = sidekit.Mixture()

ubm.EM_split(features_server,
             feature_list,
             distrib_nb,
             iterations=(1, 2, 2, 4, 4, 4, 4, 8, 8, 8, 8, 8, 8),
             num_thread=10,
             llk_gain=0.01,
             save_partial=False,
             ceil_cov=10,
             floor_cov=1e-2
             )

ubm.EM_convert_full(features_server,
                    featureList,
                    distrib_nb,
                    # EM迭代次数
                    iterations=2,
                    # 线程数
                    num_thread=10
                    )